In [29]:
import pandas as pd
import requests
import re
from geopy.geocoders import Nominatim

# Import data of Employees
employee = pd.read_csv('./data/Employee_Addresses.csv')
employee.head()

,address,employee_id
0,"98 Edinburgh St, San Francisco, CA 94112, USA",206
1,"237 Accacia St, Daly City, CA 94014, USA",2081
2,"1835 Folsom St, San Francisco, CA 94103, USA",178
3,"170 Cambridge St, San Francisco, CA 94134, USA",50
4,"16 Roanoke St, San Francisco, CA 94131, USA",1863


In [6]:
# Import data of Streets
street = pd.read_csv('./data/Potentail_Bust_Stops.csv')
street.head()

,Street_One,Street_Two
0,MISSION ST,ITALY AVE
1,MISSION ST,NEW MONTGOMERY ST
2,MISSION ST,01ST ST
3,MISSION ST,20TH ST
4,MISSION ST,FREMONT ST


In [7]:
#Pre-processing data - Create 2 new column with defaut value is None
street['lat'] = None
street['lng'] = None

In [8]:
# Get lat - lng of intersection between 2 streets

#Function to get location of intersection
def loc(street_1, street_2):
    res = requests.get('https://www.google.com/maps/place/'+street_1+' & '+street_2)
    return re.findall(r'll=(.*?)" item', res.text)[0].split(',')

In [9]:
x = loc(street['Street_One'][0],street['Street_Two'][0])
x

['37.718439', '-122.439565']

In [12]:
#Apply for dataset
for i in range(len(street)):
    street_1 = str(street['Street_One'][i])
    street_2 = str(street['Street_Two'][i])
    if street_2[0] == '0':
        street_2 = street_2[1:] + " CA"
    elif (street_2[2:4] == 'TH' and street_2[0]!='0'):
        street_2 = street_2 + " CA, USA"
    elif street_2[:4] == 'PARK':
        street_2 = street_2 + " San Francisco CA USA"
    elif (street_2[-3:] == 'AVE' and street_2[:10] != "FLORENTINE"):
        street_2 = street_2 + " San Francisco"
    elif street_2[:10] == "FLORENTINE":
        street_2 = "Florentine St"
    elif street_2[:6] == "ANGELO":
        street_2 = "Angelo's Alley"
    else:
        street_2 = street_2 + " CA"
    location = loc(street_1, street_2)
    street['lat'][i] = float(location[0])
    street['lng'][i] = float(location[1])

In [14]:
street.to_csv('./intersections.csv', index=False)

In [32]:
nom = Nominatim()

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [33]:
def get_coordinate(address):
    try:
        result = nom.geocode(address)
    
        lat = result.latitude
        lng = result.longitude
    
    except:
        lat = None
        lng = None
    
    return lat, lng

In [126]:
employee.head()

,address,employee_id,lat,lng
0,"98 Edinburgh St, San Francisco, CA 94112, USA",206,"98, Edinburgh Street, Excelsior, San Francisco, San Francisco City and County, California, 94112, United States of America","(37.727605, -122.427311)"
1,"237 Accacia St, Daly City, CA 94014, USA",2081,"237, Accacia Street, Bayshore, Daly City, San Mateo County, California, 94014, United States of America","(37.70439096918921, -122.4159151907583)"
2,"1835 Folsom St, San Francisco, CA 94103, USA",178,"1835, Folsom Street, Mission District, San Francisco, San Francisco City and County, California, 94103, United States of America","(37.76795024489796, -122.41545424489796)"
3,"170 Cambridge St, San Francisco, CA 94134, USA",50,"170, Cambridge Street, St. Mary's Park, San Francisco, San Francisco City and County, California, 94134, United States of America","(37.729671875, -122.41953875)"
4,"16 Roanoke St, San Francisco, CA 94131, USA",1863,"16, Roanoke Street, Glen Park, San Francisco, San Francisco City and County, California, 94131, United States of America","(37.7364215625, -122.43127664583334)"


In [38]:
#Pre-processing data - Create 2 new column with defaut value is None
employee['lat']=None
employee['lng']= None

#Get lat - lng of Employees' Address
for i in range(len(employee)):
    try:
        (lat, lng) = nom.geocode(employee['address'][i])
    except:
        (lat, lng) = (0,0)
    employee['lat'][i]=lat
    employee['lng'][i]=lng

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [39]:
employee.head()

,address,employee_id,lat,lng
0,"98 Edinburgh St, San Francisco, CA 94112, USA",206,"98, Edinburgh Street, Excelsior, San Francisco...","(37.727605, -122.427311)"
1,"237 Accacia St, Daly City, CA 94014, USA",2081,"237, Accacia Street, Bayshore, Daly City, San ...","(37.70439096918921, -122.4159151907583)"
2,"1835 Folsom St, San Francisco, CA 94103, USA",178,"1835, Folsom Street, Mission District, San Fra...","(37.76795024489796, -122.41545424489796)"
3,"170 Cambridge St, San Francisco, CA 94134, USA",50,"170, Cambridge Street, St. Mary's Park, San Fr...","(37.729671875, -122.41953875)"
4,"16 Roanoke St, San Francisco, CA 94131, USA",1863,"16, Roanoke Street, Glen Park, San Francisco, ...","(37.7364215625, -122.43127664583334)"


In [40]:
employee.to_csv('./em.csv', index=False)

In [106]:
errorr = employee[employee['lng'] == 0]
errorr.head()

,address,employee_id,lat,lng
240,"St. Luke's Hospital Garage, San Francisco, CA ...",941,0,0
313,"St. Luke's Hospital Garage, San Francisco, CA ...",1320,0,0
474,San Francisco War Memorial and Performing Arts...,1814,0,0
480,"141 Treat St, San Francisco, CA 94103, USA",647,0,0
764,"3119 Pierce St, San Francisco, CA 94123, USA",1239,0,0


In [81]:
errorr['address'] = errorr[['address']].apply(lambda x: x[0].partition(', CA')[0],axis=1)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [71]:
errorr[['address']].apply(lambda x: x[0].partition(', CA')[0],axis=1)

240             St. Luke's Hospital Garage, San Francisco
313             St. Luke's Hospital Garage, San Francisco
474     San Francisco War Memorial and Performing Arts...
480                           141 Treat St, San Francisco
764                         3119 Pierce St, San Francisco
                              ...                        
2186                         72 Topeka Ave, San Francisco
2187                             330 Oliver St, Daly City
2188                         94 W View Ave, San Francisco
2189                       1599 Geneva Ave, San Francisco
2190                         499 Treat Ave, San Francisco
Length: 995, dtype: object

In [77]:
#Pre-processing data - Create 2 new column with defaut value is None
errorr['lat']=None
errorr['lng']= None

#Get lat - lng of Employees' Address
for i in range(len(errorr)):
    try:
        (lat, lng) = nom.geocode(errorr['address'][i])
    except:
        (lat, lng) = (0,0)
    errorr['lat'][i]=lat
    errorr['lng'][i]=lng

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

In [79]:
errorr.to_csv('./errorr.csv', index=False)

In [83]:
errorr.head()

,address,employee_id,lat,lng
240,"St. Luke's Hospital Garage, San Francisco, CA ...",941,0,0
313,"St. Luke's Hospital Garage, San Francisco, CA ...",1320,0,0
474,San Francisco War Memorial and Performing Arts...,1814,0,0
480,"141 Treat St, San Francisco, CA 94103, USA",647,0,0
764,"3119 Pierce St, San Francisco, CA 94123, USA",1239,0,0


In [121]:
errorr['address'] = errorr[['address']].apply(lambda x: str(x).replace(', CA ', ', California, '), axis=1)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [125]:
pd.set_option('display.max_colwidth', -1)
errorr.head()

,address,employee_id,lat,lng
240,"address St. Luke's Hospital Garage, San Francisco, California, ...\nName: 240, dtype: object",941,0,0
313,"address St. Luke's Hospital Garage, San Francisco, California, ...\nName: 313, dtype: object",1320,0,0
474,"address San Francisco War Memorial and Performing Arts...\nName: 474, dtype: object",1814,0,0
480,"address 141 Treat St, San Francisco, California, 94103, USA\nName: 480, dtype: object",647,0,0
764,"address 3119 Pierce St, San Francisco, California, 94123, USA\nName: 764, dtype: object",1239,0,0


In [109]:
'141 Treat St, San Francisco, CA 94103, USA'.replace(', CA ', ', California, ')

'141 Treat St, San Francisco, California, 94103, USA'

In [119]:
errorr.dtypes

address        object
employee_id     int64
lat            object
lng            object
dtype: object